In [1]:
### 데이터 전처리
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df
def drop_features(df):
    df.drop(columns = ['PassengerId','Name','Ticket'],inplace=True) # 필요없는 열 제거
    return df
def format_features(df):
    from sklearn.preprocessing import LabelEncoder
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin','Sex','Embarked']
    for feature in features: # 각 열의 레이블을 만들고 학습
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature]) # 변경된 것을 원래 위치에 넣기
        print(le.classes_)
    return df
def transform_features():
    import pandas as pd
    df = pd.read_csv('titanic_train.csv')
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [2]:
titanic_df = transform_features()

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
['female' 'male']
['C' 'N' 'Q' 'S']


In [3]:
titanic_df.head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.25,7,3


In [4]:
### 학습 데이터 검증 데이터 나누기
from sklearn.model_selection import train_test_split

In [5]:
X_df = titanic_df.drop(columns=['Survived'])
# X에는 결과값(Survived)을 빼고 넣으면 됨
# drop이어도 원본 데이터에는 영향주지 x고 X_df에 따로 저장함 
y_df = titanic_df['Survived']

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=11)

In [7]:
### 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

In [9]:
dt_clf.fit(X_train,y_train)
dt_pred = dt_clf.predict(X_test)
print(f'DecisionTreeClassifier 정확도 : {accuracy_score(y_test,dt_pred):.4f}') # 소수 이하 4자리 실수값으로

DecisionTreeClassifier 정확도 : 0.7877


In [10]:
rf_clf.fit(X_train,y_train)
rf_pred = rf_clf.predict(X_test)
print(f'RandomForestClassifierr 정확도 : {accuracy_score(y_test,rf_pred):.4f}')

RandomForestClassifierr 정확도 : 0.8547


In [11]:
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
print(f'LogisticRegression 정확도 : {accuracy_score(y_test,lr_pred):.4f}')

LogisticRegression 정확도 : 0.8659


In [12]:
from sklearn.model_selection import KFold

In [13]:
def exec_kfold(clf,fold=5):
    import numpy as np
    kfold = KFold(n_splits=fold)
    scores =[]
    for iter_count,(train_index,test_index) in enumerate(kfold.split(X_df)) :
        X_train,X_test = X_df.values[train_index],X_df.values[test_index]
        y_train,y_test = y_df.values[train_index],y_df.values[test_index]
    # 데이터값이랑 인덱스가 같이 필요할 때 사용
        clf.fit(X_train,y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test,pred)
        scores.append(accuracy)
        print(f'교차검증 {iter_count} 정확도 : {accuracy}')
    print(f'평균정확도 : {np.mean(scores)}')

In [14]:
exec_kfold(dt_clf)  # DecisionTreeClassifier 정확도 : 0.7877

교차검증 0 정확도 : 0.7541899441340782
교차검증 1 정확도 : 0.7808988764044944
교차검증 2 정확도 : 0.7865168539325843
교차검증 3 정확도 : 0.7696629213483146
교차검증 4 정확도 : 0.8202247191011236
평균정확도 : 0.782298662984119


In [31]:
### 교차검증
exec_kfold(rf_clf) # RandomForestClassifierr 정확도 : 0.8547

교차검증 0 정확도 : 0.7932960893854749
교차검증 1 정확도 : 0.8089887640449438
교차검증 2 정확도 : 0.8370786516853933
교차검증 3 정확도 : 0.7752808988764045
교차검증 4 정확도 : 0.8595505617977528
평균정확도 : 0.8148389931579938


In [16]:
exec_kfold(lr_clf)  # LogisticRegression 정확도 : 0.8659

교차검증 0 정확도 : 0.7932960893854749
교차검증 1 정확도 : 0.7921348314606742
교차검증 2 정확도 : 0.7752808988764045
교차검증 3 정확도 : 0.7471910112359551
교차검증 4 정확도 : 0.8426966292134831
평균정확도 : 0.7901198920343984


In [17]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [35]:
score = cross_val_score(dt_clf,X_df,y_df, cv=5)
print(f'{score} 평균 {np.mean(score)}')
# 교차검증 0 정확도 : 0.7541899441340782
# 교차검증 1 정확도 : 0.7808988764044944
# 교차검증 2 정확도 : 0.7865168539325843
# 교차검증 3 정확도 : 0.7696629213483146
# 교차검증 4 정확도 : 0.8202247191011236
# 평균정확도 : 0.782298662984119

[0.74301676 0.7752809  0.79213483 0.78651685 0.84269663] 평균 0.7879291946519366


In [36]:
score = cross_val_score(rf_clf,X_df,y_df, cv=5)
print(f'{score} 평균 {np.mean(score)}')
# 교차검증 0 정확도 : 0.7932960893854749
# 교차검증 1 정확도 : 0.8089887640449438
# 교차검증 2 정확도 : 0.8370786516853933
# 교차검증 3 정확도 : 0.7752808988764045
# 교차검증 4 정확도 : 0.8595505617977528
# 평균정확도 : 0.8148389931579938

[0.79329609 0.79775281 0.84831461 0.76404494 0.86516854] 평균 0.8137153976523758


In [37]:
score = cross_val_score(lr_clf,X_df,y_df, cv=5)
print(f'{score} 평균 {np.mean(score)}')
# 교차검증 0 정확도 : 0.7932960893854749
# 교차검증 1 정확도 : 0.7921348314606742
# 교차검증 2 정확도 : 0.7752808988764045
# 교차검증 3 정확도 : 0.7471910112359551
# 교차검증 4 정확도 : 0.8426966292134831
# 평균정확도 : 0.7901198920343984

[0.7877095  0.79213483 0.7752809  0.76404494 0.81460674] 평균 0.7867553825874082


In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
parameter = {
    'max_depth':[2,3,5,10],
    'min_samples_split':[2,3,5],
    'min_samples_leaf':[1,5,8],
}

In [22]:
grid_clf = GridSearchCV(dt_clf,param_grid=parameter,scoring='accuracy',cv=5)
grid_clf.fit(X_train,y_train) # 학습

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=11),
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 8],
                         'min_samples_split': [2, 3, 5]},
             scoring='accuracy')

In [23]:
grid_clf.best_params_  # 뭐가 가장 좋았는지

{'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}

In [24]:
grid_clf.best_score_

0.7991825076332119

In [25]:
pred = grid_clf.best_estimator_.predict(X_test) # 예측

In [26]:
accuracy_score(y_test,pred) # 파라미터 튜닝 후 예측값

0.8715083798882681

In [27]:
grid_clf = GridSearchCV(rf_clf,param_grid=parameter,scoring='accuracy',cv=5)
grid_clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=11),
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 8],
                         'min_samples_split': [2, 3, 5]},
             scoring='accuracy')

In [28]:
grid_clf.best_params_

{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [29]:
grid_clf.best_score_

0.8145966709346991

In [30]:
pred = grid_clf.best_estimator_.predict(X_test)
accuracy_score(y_test,pred)

0.88268156424581

In [ ]:
# 평가 방법에 정확도만을 적용할 수 없는 이유 
# 이진 분류인데 데이터가 한 쪽으로 쏠려 불균형한 경우 정확도가 평가척도로서 잘 기능하지 못함